# Multi-echo demo in Neurodesk: fMRIPrep → tedana (ds005123, sub-10317)

Goal: run a simple, readable **end-to-end** demo on **one subject**.

Notes (important for expectations):
- `--me-output-echos` makes fMRIPrep write **per-echo preprocessed** time series for downstream tedana.
- Those per-echo outputs are **not named with `space-MNI...`** (they are in the preprocessed BOLD “native” space). fMRIPrep separately resamples the *combined* outputs to your requested standard spaces.


## 0) Set paths

In [ ]:
import os
from pathlib import Path

SUB = "10317"

# Change if you want a persistent location (e.g., /neurodesktop-storage/...)
PROJ = Path.home() / "ds005123_me_demo"

BIDS = PROJ / "bids"
FMRIPREP_OUT = PROJ / "derivatives" / "fmriprep-25.2.3"
WORK = PROJ / "work"
TEDANA_OUT = PROJ / "derivatives" / "tedana"
LICENSES = PROJ / "licenses"

for p in [PROJ, BIDS, FMRIPREP_OUT, WORK, TEDANA_OUT, LICENSES]:
    p.mkdir(parents=True, exist_ok=True)

print("PROJ:", PROJ)
print("BIDS:", BIDS)
print("FMRIPREP_OUT:", FMRIPREP_OUT)
print("TEDANA_OUT:", TEDANA_OUT)


## 1) Load fMRIPrep (Neurodesk module)

If `lmod` is available (as in Neurodesk examples), we use it. Otherwise we fall back to `module`.


In [ ]:
# Neurodesk-style module loading
try:
    import lmod as _mod
except Exception:
    import module as _mod

await _mod.load("fmriprep/25.2.3")
await _mod.list()

import shutil, subprocess
print("which fmriprep:", shutil.which("fmriprep"))
subprocess.run(["fmriprep", "--version"], check=False)


## 2) Download ds005123 and get sub-10317 (DataLad)

This prints directly to the notebook.


In [ ]:
%cd {PROJ}
!datalad -l error install -s https://github.com/OpenNeuroDatasets/ds005123.git bids

%cd {BIDS}
!datalad -l error get sub-{SUB}

!ls -1 {BIDS}/sub-{SUB}/func


## 3) FreeSurfer license

This demo expects your license at `~/.license` and copies it into the project.


In [ ]:
!test -r ~/.license
!mkdir -p {LICENSES}
!cp -f ~/.license {LICENSES}/fs_license.txt
!ls -l {LICENSES}/fs_license.txt


## 4) Run fMRIPrep (MNI152NLin6Asym only + per-echo outputs)

This is the command you care about, without logging/pipefail tricks.


In [ ]:
import os
from pathlib import Path

# Create a real subjects dir to avoid the FreeSurfer TraitError
subjects_dir = Path(FMRIPREP_OUT) / "sourcedata" / "freesurfer"
subjects_dir.mkdir(parents=True, exist_ok=True)

# Ensure fMRIPrep's container sees SUBJECTS_DIR too
os.environ["SUBJECTS_DIR"] = str(subjects_dir)
os.environ["APPTAINERENV_SUBJECTS_DIR"] = str(subjects_dir)

print("SUBJECTS_DIR:", os.environ["SUBJECTS_DIR"])


In [ ]:
!mkdir -p {FMRIPREP_OUT} {WORK}

!fmriprep "{BIDS}" "{FMRIPREP_OUT}" participant   --participant-label "{SUB}"   --stop-on-first-crash   --skip-bids-validation   --me-output-echos   --output-spaces MNI152NLin6Asym   --fs-no-reconall   --fs-license-file "{LICENSES}/fs_license.txt"   --fs-subjects-dir "{FMRIPREP_OUT}/sourcedata/freesurfer"   -w "{WORK}"   --nthreads 14 --omp-nthreads 1 --mem-mb 24000   -v


### Quick check: fMRIPrep outputs

In [ ]:
!ls -l {FMRIPREP_OUT}/sub-{SUB}.html
!ls -1 {FMRIPREP_OUT}/sub-{SUB}/func


## 5) Find echo-wise outputs + EchoTimes (TEs)

We do **not** assume `part-mag` or `space-MNI...` in the echo-wise filenames. We just find what fMRIPrep actually wrote.


In [ ]:
import json
from pathlib import Path

RUN = "task-doors_run-1"  # change if you want a different run

funcdir = Path(FMRIPREP_OUT) / f"sub-{SUB}" / "func"

# Find an echo-1 file for this run (allowing optional entities like part-mag)
echo1 = sorted(funcdir.glob(f"sub-{SUB}_{RUN}_echo-1*desc-preproc_bold.nii.gz"))
if not echo1:
    available = sorted(funcdir.glob(f"sub-{SUB}_*echo-*desc-preproc_bold.nii.gz"))
    print("No echo-1 preproc file found for", RUN)
    print("Echo-wise preproc files that DO exist:")
    for p in available:
        print(" ", p.name)
    raise RuntimeError("Update RUN to match an existing run above.")

echo1 = echo1[0]
print("Using echo-1 file:")
print(" ", echo1.name)

# Build an all-echo pattern by swapping '_echo-1' -> '_echo-*'
all_echo_pat = echo1.name.replace("_echo-1", "_echo-*")
ECHOS = sorted(funcdir.glob(all_echo_pat))

print("\nEcho-wise inputs to tedana:")
for p in ECHOS:
    print(" ", p.name)

# Match raw JSONs to get EchoTimes
raw_funcdir = Path(BIDS) / f"sub-{SUB}" / "func"
raw_json_echo1 = sorted(raw_funcdir.glob(f"sub-{SUB}_{RUN}_echo-1*bold.json"))
if not raw_json_echo1:
    raise RuntimeError(f"Could not find raw JSON sidecars for {RUN} (echo-1).")

raw_json_echo1 = raw_json_echo1[0]
raw_all_pat = raw_json_echo1.name.replace("_echo-1", "_echo-*")
RAW_JSONS = sorted(raw_funcdir.glob(raw_all_pat))

TEs = []
print("\nEchoTimes (seconds) from raw JSONs:")
for jp in RAW_JSONS:
    with open(jp, "r") as f:
        te = float(json.load(f)["EchoTime"])
    TEs.append(te)
    print(" ", jp.name, "->", te)


## 6) Install tedana (no module) and run it

We keep this simple:
- `pip install --user tedana`
- run tedana via `subprocess` (avoids fragile shell strings)


In [ ]:
!python3 -m pip install --user tedana


In [ ]:
import os, subprocess
from pathlib import Path

# Ensure ~/.local/bin is on PATH (pip --user installs here)
os.environ["PATH"] = str(Path.home() / ".local" / "bin") + ":" + os.environ.get("PATH", "")

subprocess.run(["which", "tedana"], check=False)
subprocess.run(["tedana", "--version"], check=False)

outdir = Path(TEDANA_OUT) / f"sub-{SUB}" / "func" / f"sub-{SUB}_{RUN}_tedana"
outdir.mkdir(parents=True, exist_ok=True)

cmd = [
    "tedana",
    "-d", *[str(p) for p in ECHOS],
    "-e", *[str(te) for te in TEs],
    "--convention", "bids",
    "--out-dir", str(outdir),
    "--prefix", f"sub-{SUB}_{RUN}",
    "--fittype", "curvefit",
    "--overwrite",
]

print("Running tedana (command shortened):")
print("tedana -d <echo files> -e <TEs> ...")
subprocess.run(cmd, check=True)

print("\nDone. Example outputs:")
for p in sorted(outdir.iterdir())[:25]:
    print(" ", p.name)
